# Install All dependencies

In [ ]:
! pip install faiss-gpu

In [ ]:
! git clone https://github.com/zhengxxn/adaptive-knn-mt.git
! ls

In [ ]:
! pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu102.html

In [ ]:
! pip install sacremoses
! pip install sacrebleu==1.4.12

In [ ]:
! pip install -e adaptive-knn-mt/
! pip install tensorboardX

# Generate Data Store

In [ ]:
# it
%env DSTORE_SIZE=3613350
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
%env DATA_PATH=it
%env DATASTORE_PATH=it_data_store
%env PROJECT_PATH=adaptive-knn-mt

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/save_datastore.py $DATA_PATH \
    --dataset-impl mmap \
    --task translation \
    --valid-subset train \
    --path $MODEL_PATH \
    --max-tokens 4096 --skip-invalid-size-inputs-valid-test \
    --decoder-embed-dim 1024 --dstore-fp16 --dstore-size $DSTORE_SIZE --dstore-mmap $DATASTORE_PATH
 
# 4096 and 1024 depend on your device and model separately

In [ ]:
# medical
%env DSTORE_SIZE=6903320
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
%env DATA_PATH=medical
%env DATASTORE_PATH=medical_data_store
%env PROJECT_PATH=adaptive-knn-mt

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/save_datastore.py $DATA_PATH \
    --dataset-impl mmap \
    --task translation \
    --valid-subset train \
    --path $MODEL_PATH \
    --max-tokens 4096 --skip-invalid-size-inputs-valid-test \
    --decoder-embed-dim 1024 --dstore-fp16 --dstore-size $DSTORE_SIZE --dstore-mmap $DATASTORE_PATH
 
# 4096 and 1024 depend on your device and model separately

In [ ]:
# koran
%env DSTORE_SIZE=524400
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
%env DATA_PATH=koran
%env DATASTORE_PATH=koran_data_store
%env PROJECT_PATH=adaptive-knn-mt

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/save_datastore.py $DATA_PATH \
    --dataset-impl mmap \
    --task translation \
    --valid-subset train \
    --path $MODEL_PATH \
    --max-tokens 4096 --skip-invalid-size-inputs-valid-test \
    --decoder-embed-dim 1024 --dstore-fp16 --dstore-size $DSTORE_SIZE --dstore-mmap $DATASTORE_PATH
 
# 4096 and 1024 depend on your device and model separately

In [ ]:
# law
%env DSTORE_SIZE=19070000
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
%env DATA_PATH=law
%env DATASTORE_PATH=law_data_store
%env PROJECT_PATH=adaptive-knn-mt

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/save_datastore.py $DATA_PATH \
    --dataset-impl mmap \
    --task translation \
    --valid-subset train \
    --path $MODEL_PATH \
    --max-tokens 4096 --skip-invalid-size-inputs-valid-test \
    --decoder-embed-dim 1024 --dstore-fp16 --dstore-size $DSTORE_SIZE --dstore-mmap $DATASTORE_PATH
 
# 4096 and 1024 depend on your device and model separately

# Genreate Faiss Index

In [ ]:
# it
%env PROJECT_PATH=adaptive-knn-mt
%env DSTORE_PATH=it_data_store
%env DSTORE_SIZE=3613350

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/train_datastore_gpu.py \
    --dstore_mmap $DSTORE_PATH \
    --dstore_size $DSTORE_SIZE \
    --dstore-fp16 \
    --faiss_index ${DSTORE_PATH}/knn_index \
    --ncentroids 4096 \
    --probe 32 \
    --dimension 1024

In [ ]:
# medical
%env PROJECT_PATH=adaptive-knn-mt
%env DSTORE_PATH=medical_data_store
%env DSTORE_SIZE=6903320

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/train_datastore_gpu.py \
    --dstore_mmap $DSTORE_PATH \
    --dstore_size $DSTORE_SIZE \
    --dstore-fp16 \
    --faiss_index ${DSTORE_PATH}/knn_index \
    --ncentroids 4096 \
    --probe 32 \
    --dimension 1024

In [ ]:
# koran
%env PROJECT_PATH=adaptive-knn-mt
%env DSTORE_PATH=koran_data_store
%env DSTORE_SIZE=524400

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/train_datastore_gpu.py \
    --dstore_mmap $DSTORE_PATH \
    --dstore_size $DSTORE_SIZE \
    --dstore-fp16 \
    --faiss_index ${DSTORE_PATH}/knn_index \
    --ncentroids 4096 \
    --probe 32 \
    --dimension 1024

In [ ]:
# law
%env PROJECT_PATH=adaptive-knn-mt
%env DSTORE_PATH=law_data_store
%env DSTORE_SIZE=19070000

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/train_datastore_gpu.py \
    --dstore_mmap $DSTORE_PATH \
    --dstore_size $DSTORE_SIZE \
    --dstore-fp16 \
    --faiss_index ${DSTORE_PATH}/knn_index \
    --ncentroids 4096 \
    --probe 32 \
    --dimension 1024

# Training

In [ ]:
# Adjust the training script according to the description in the README
! bash train.sh

# Inference for IT domain
This is just part of our code. The code for inference for each domain is pretty similar. We just reused our code by changing the parameters as the way we stated in the README file.

In [ ]:
%env DSTORE_SIZE=3613350
%env MODEL_PATH=model_record/train-hid32-maxk1/checkpoint.best_loss_1.97.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=it
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=output

! mkdir -p "$OUTPUT_PATH"

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH \
--gen-subset test --path $MODEL_PATH \
--arch transformer_wmt19_de_en_with_datastore \
--beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en \
--scoring sacrebleu \
--batch-size 32 \
--tokenizer moses --remove-bpe \
--model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True, \
'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, \
'dstore_fp16': True, 'probe': 32,'knn_sim_func': 'do_not_recomp_l2', \
'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True, \
'knn_temperature_type': 'fix', 'knn_temperature_value': 10,}" \
| tee "$OUTPUT_PATH"/generate.txt

!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok

In [ ]:
%env DSTORE_SIZE=3613350
%env MODEL_PATH=model_record/train-hid32-maxk2/checkpoint.best_loss_1.88.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=it
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=output

! mkdir -p "$OUTPUT_PATH"

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH \
--gen-subset test --path $MODEL_PATH \
--arch transformer_wmt19_de_en_with_datastore \
--beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en \
--scoring sacrebleu \
--batch-size 32 \
--tokenizer moses --remove-bpe \
--model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True, \
'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, \
'dstore_fp16': True, 'probe': 32,'knn_sim_func': 'do_not_recomp_l2', \
'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True, \
'knn_temperature_type': 'fix', 'knn_temperature_value': 10,}" \
| tee "$OUTPUT_PATH"/generate.txt

!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok

In [ ]:
%env DSTORE_SIZE=3613350
%env MODEL_PATH=model_record/train-hid32-maxk4/checkpoint.best_loss_1.83.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=it
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=output

! mkdir -p "$OUTPUT_PATH"

! CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH \
--gen-subset test --path $MODEL_PATH \
--arch transformer_wmt19_de_en_with_datastore \
--beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en \
--scoring sacrebleu \
--batch-size 32 \
--tokenizer moses --remove-bpe \
--model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True, \
'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, \
'dstore_fp16': True, 'probe': 32,'knn_sim_func': 'do_not_recomp_l2', \
'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True, \
'knn_temperature_type': 'fix', 'knn_temperature_value': 10,}" \
| tee "$OUTPUT_PATH"/generate.txt

!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok

In [ ]:
# Base

%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt
%env DATA_PATH=it
%env DATASTORE_PATH=data_store
%env PROJECT_PATH=adaptive-knn-mt

!mkdir -p $OUTPUT_PATH

!CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/fairseq_cli/generate.py $DATA_PATH\
    --gen-subset test \
    --path $MODEL_PATH \
    --beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en \
    --scoring sacrebleu \
    --max-tokens 4096 \
    --tokenizer moses --remove-bpe | tee $OUTPUT_PATH/generate.txt

!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok

In [ ]:
# Vanilla

%env DSTORE_SIZE=3613350
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=it
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=output

!mkdir -p "$OUTPUT_PATH"

!CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH\
    --gen-subset test\
    --path $MODEL_PATH --arch transformer_wmt19_de_en_with_datastore\
    --beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en\
    --scoring sacrebleu\
    --batch-size 32\
    --tokenizer moses --remove-bpe\
    --model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True,'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, 'dstore_fp16': True, 'k': 1, 'probe': 32,'knn_sim_func': 'do_not_recomp_l2', 'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True,'knn_lambda_type': 'fix', 'knn_lambda_value': 0.7, 'knn_temperature_type': 'fix', 'knn_temperature_value': 10,}" \
    | tee "$OUTPUT_PATH"/generate.txt
!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok


In [ ]:
%env DSTORE_SIZE=3613350
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=it
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=output

!mkdir -p "$OUTPUT_PATH"

!CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH\
    --gen-subset test\
    --path $MODEL_PATH --arch transformer_wmt19_de_en_with_datastore\
    --beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en\
    --scoring sacrebleu\
    --batch-size 32\
    --tokenizer moses --remove-bpe\
    --model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True,'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, 'dstore_fp16': True, 'k': 2, 'probe': 32,'knn_sim_func': 'do_not_recomp_l2', 'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True,'knn_lambda_type': 'fix', 'knn_lambda_value': 0.7, 'knn_temperature_type': 'fix', 'knn_temperature_value': 10,}" \
    | tee "$OUTPUT_PATH"/generate.txt
!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok


In [ ]:
%env DSTORE_SIZE=3613350
%env MODEL_PATH=Model/wmt19.de-en.ffn8192.pt

%env DATASTORE_PATH=data_store
%env DATA_PATH=it
%env PROJECT_PATH=adaptive-knn-mt

%env OUTPUT_PATH=output

!mkdir -p "$OUTPUT_PATH"

!CUDA_VISIBLE_DEVICES=0 python $PROJECT_PATH/experimental_generate.py $DATA_PATH\
    --gen-subset test\
    --path $MODEL_PATH --arch transformer_wmt19_de_en_with_datastore\
    --beam 4 --lenpen 0.6 --max-len-a 1.2 --max-len-b 10 --source-lang de --target-lang en\
    --scoring sacrebleu\
    --batch-size 32\
    --tokenizer moses --remove-bpe\
    --model-overrides "{'load_knn_datastore': True, 'use_knn_datastore': True,'dstore_filename': '$DATASTORE_PATH', 'dstore_size': $DSTORE_SIZE, 'dstore_fp16': True, 'k': 4, 'probe': 32,'knn_sim_func': 'do_not_recomp_l2', 'use_gpu_to_search': True, 'move_dstore_to_mem': True, 'no_load_keys': True,'knn_lambda_type': 'fix', 'knn_lambda_value': 0.7, 'knn_temperature_type': 'fix', 'knn_temperature_value': 10,}" \
    | tee "$OUTPUT_PATH"/generate.txt
!grep ^S "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/src
!grep ^T "$OUTPUT_PATH"/generate.txt | cut -f2- > "$OUTPUT_PATH"/ref
!grep ^H "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp
!grep ^D "$OUTPUT_PATH"/generate.txt | cut -f3- > "$OUTPUT_PATH"/hyp.detok